# 0. Import des packages et des données 

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
from helpers2 import S3Connection

In [ ]:
from postal.expand import expand_address

In [ ]:
s3 = S3Connection(bucket_name="clichere/diffusion")

In [ ]:
path1 = "DPE/DPE_logements.parquet"
DPE = s3.get_tables_from_s3(path1)

In [ ]:
path_vf_2024 = "valeursfoncieres/vf_2024.csv"
vf_2024 = s3.read_csv_from_s3(path_vf_2024)

path_vf_2023 = "valeursfoncieres/vf_2023.csv"
vf_2023 = s3.read_csv_from_s3(path_vf_2023)

path_vf_2022 = "valeursfoncieres/vf_2022.csv"
vf_2022 = s3.read_csv_from_s3(path_vf_2022)

path_vf_2021 = "valeursfoncieres/vf_2021.csv"
vf_2021 = s3.read_csv_from_s3(path_vf_2021)

path_vf_2020 = "valeursfoncieres/vf_2020.csv"
vf_2020 = s3.read_csv_from_s3(path_vf_2020)

path_vf_2019 = "valeursfoncieres/vf_2019.csv"
vf_2019 = s3.read_csv_from_s3(path_vf_2019)

# 1. FONCTIONS UTILES

Normalisation des adresses du fichier DPE et valeurs foncières. 

In [ ]:
def normalize_address(address):
    if pd.isna(address) or address.strip() == '':
        return None  
    try:
        normalized = expand_address(address)  
        return normalized[0] if normalized else None  # Ne garde que la première version de la normalisation
    except Exception as e:
        print(f"Erreur avec l'adresse '{address}': {e}")
        return None

In [ ]:
import re
def normalize_vf_address(address, code_postal):
    """ Normalise l'adresse et supprime l'arrondissement après 'PARIS' uniquement si le département est 75. """
    normalized_address = normalize_address(address)  

    if pd.notna(normalized_address) and str(code_postal).startswith("75"):
        # Supprime le numéro après "PARIS" uniquement pour le département 75
        normalized_address = re.sub(r'(paris) \d{2}$', r'\1', normalized_address, flags=re.IGNORECASE)
    
    return normalized_address

FONCTION DE MATCHING

In [ ]:
def test_match(vf, df):
    merged = []


    #boucle par département
    for department in df['N°_département_(BAN)'].unique():  
        print(f"Traitement du département : {department}")
        
        #filtre
        vf_dept = vf[vf['code_departement']==department].copy()
        df_dept = df[df['N°_département_(BAN)']==department].copy()

        #normalisation adresses
        vf_dept['Adresse'] = vf_dept['Adresse'].str.strip().str.replace(r'\s+', ' ', regex=True)
        vf_dept['Adresse_Normalisee'] = vf_dept.apply(lambda row: normalize_vf_address(row['Adresse'], row['code_departement']), axis=1)
        df_dept['Adresse_Normalisee'] = df_dept['Adresse_(BAN)'].apply(normalize_address)

        #On distingue les adresses uniques et les adresses en doublons pour avoir deux catégories d'adresses et trouver celles communes aux deux bases
        adresse_counts = vf_dept['Adresse_Normalisee'].dropna().value_counts()
        unique1 = list(adresse_counts[adresse_counts == 1].index)
        doublons1 = list(adresse_counts[adresse_counts > 1].index)
        final = unique1 + doublons1
        set_final = set(final)

        adresse_counts2 = df_dept['Adresse_Normalisee'].dropna().value_counts()
        unique2 = list(adresse_counts2[adresse_counts2 == 1].index)
        doublons2 = list(adresse_counts2[adresse_counts2 > 1].index)
        final2 = unique2 + doublons2
        set_final2 = set(final2)

        # On regarde les adresses communes
        commun = set_final.intersection(set_final2)

        vf_dept['surface_reelle_bati'] = pd.to_numeric(
            vf_dept['surface_reelle_bati'].astype(str).str.replace(',', '.'), errors='coerce'
        )
        df_dept['Surface_habitable_logement'] = pd.to_numeric(
            df_dept['Surface_habitable_logement'].astype(str).str.replace(',', '.'), errors='coerce'
        )

        for adresse in commun:
            # On crée des sous dataframe contenant les lignes avec les mêmes adresses
            dfsub = df_dept[df_dept['Adresse_Normalisee'] == adresse]
            vfsub = vf_dept[vf_dept['Adresse_Normalisee'] == adresse]

            # Boucle sur les rangs des sous dataframe
            for _, row2 in dfsub.iterrows():
                best_match = None
                best_value = -1

                for _, row1 in vfsub.iterrows():
                    surface1 = row1['surface_reelle_bati']
                    surface2 = row2['Surface_habitable_logement']

                    #si surfaces identiques on match direct
                    if surface1 == surface2:
                        best_match = row1
                        break

                    #ecart inf à 5% et parmi les lignes avec des surfaces inf au seuil si jamais la valeur foncière est supérieure à celle de la ligne d'avant on la conserve pour avoir la plus grande
                    if abs(surface1 - surface2) / max(surface1, surface2) < 0.05:
                        valeur_fonciere = pd.to_numeric(str(row1.get('valeur_fonciere', 0)).replace(',', '.'), errors='coerce')
                        if valeur_fonciere > best_value:
                            best_value = valeur_fonciere
                            best_match = row1

                if best_match is not None:
                    merged.append({**row2.to_dict(), **best_match.to_dict()})

    #df des résultats fusionnés
    df = pd.DataFrame(merged)
    return df.drop_duplicates(subset=['Date_établissement_DPE', 'Adresse_Normalisee', 'Surface_habitable_logement', 'valeur_fonciere', 'Etiquette_DPE'])


In [ ]:
def compiler_dataframes(df1, df2):
    #concatène les deux DataFrames
    df_compilé = pd.concat([df1, df2], ignore_index=True)
    #supprime doublons
    df_compilé = df_compilé.drop_duplicates()
    return df_compilé

Fonction pour mettre en forme les fichiers valeurs foncières.

In [ ]:
def format_vf(vf):
    # On crée la colonne adresse
    vf['Adresse'] = vf['adresse_numero'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['adresse_nom_voie'].fillna('').astype(str) + " " + \
                vf['code_postal'].apply(lambda x: str(int(x)) if pd.notna(x) else '').astype(str) + " " + \
                vf['nom_commune'].fillna('').astype(str)

    #On formate
    vf['Adresse'] = vf['Adresse'].str.strip().replace(r'^\s*$', None, regex=True)  # Supprime les adresses vides

    vf = vf[vf['code_departement'].notna()]
    vf['code_departement'] = vf['code_departement'].astype(str)

    return vf

Fonction pour mettre en forme le fichier DPE. 

In [ ]:
def format_dpe(df,annee):
    df['Date_établissement_DPE'] = pd.to_datetime(df['Date_établissement_DPE'], errors='coerce')
    df = df[df['Date_établissement_DPE'].dt.year == annee].copy()
    df = df[df['N°_département_(BAN)'].notna()]
    df.loc[:, 'N°_département_(BAN)'] = df['N°_département_(BAN)'].astype(str)

    return df

# 2. Année 2019

In [ ]:
v2019 = format_vf(vf_2019)

In [ ]:
print(vf_2019.columns)

In [ ]:
d2019 = format_dpe(DPE, 2019)

In [ ]:
print(DPE.columns)